In [1]:
from jupyter_cadquery import set_defaults, open_viewer

set_defaults(theme="dark")
open_viewer("CadQuery")

import math
import cadquery as cq
from cadquery import Vector
from cq_gears import SpurGear, RingGear
from jupyter_cadquery.viewer.client import show

Overwriting auto display for cadquery Workplane and Shape


In [ ]:
main_arm_length = 150
secondary_arm_length = main_arm_length

arm_height = 10
arm_thickness = 5
arm_extra_length = 0

gear_size = main_arm_length
gear_height = 5

In [2]:
# secondary_motor = cq.importers.importStep("nema17.step").rotate((0,0,0),(1,0,0),90)

main_arm = (
    cq.Workplane("XY")
    .workplane(-gear_height)
    .center(main_arm_length / 2 - main_arm_length / 3 / 2, 0)
    .rect(main_arm_length + arm_extra_length * 2, arm_thickness * 2)
    .extrude(arm_height)
)

secondary_arm = (
    main_arm.faces(">Z")
    .workplane(gear_height)
    .center(gear_size, -arm_thickness)
    .rect(
        secondary_arm_length + arm_extra_length * 2, arm_thickness * 2, centered=False
    )
    .extrude(arm_height)
)

arm = main_arm.union(secondary_arm)

: 

In [ ]:
gear_arm_forward = SpurGear(
    module=1.0, teeth_number=gear_size, width=gear_height, helix_angle=30
)
gear_arm_backward = SpurGear(
    module=1.0, teeth_number=gear_size, width=gear_height, helix_angle=-30
)

ring_gear_rim_width = 10
gear_motor_main = RingGear(
    module=1.0,
    teeth_number=gear_size - ring_gear_rim_width,
    rim_width=ring_gear_rim_width,
    width=gear_height,
    helix_angle=30,
)
gear_motor_secondary = SpurGear(
    module=1.0,
    teeth_number=math.floor(gear_size / 10),
    width=gear_height,
    helix_angle=30,
)

arm_gear_1 = (
    main_arm.faces(">Z")
    .workplane()
    .center(-gear_size, 0)
    .circle(gear_arm_forward.r0)
    .extrude(gear_arm_forward.width)
)
# .gear(gear_arm_forward))

arm_gear_2 = (
    main_arm.faces(">Z")
    .workplane()
    .circle(gear_arm_backward.r0)
    .extrude(gear_arm_backward.width)
)
# .gear(gear_arm_backward))

ring_gear = (
    arm_gear_1.faces("<Z[1]")
    .workplane()
    .circle(gear_motor_main.r0)
    .extrude(gear_motor_main.width)
)
# .gear(gear_motor_main))

gears = arm_gear_1.union(arm_gear_2).union(ring_gear)

In [ ]:
main_motor = cq.importers.importStep("nema17.step").rotate((0, 0, 0), (1, 0, 0), 90)
motor_size = main_motor.val().BoundingBox().xlen
main_motor = main_motor.translate([-motor_size / 2, 0, 0])

In [ ]:
final = arm.union(gears).union(main_motor)

show(final, grid=[True, True, True])